# Week 11 -- Data Cleaning

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
import warnings
warnings.simplefilter('ignore')

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.1f' % x)

## Functions

In [3]:
def check_names(df1, df2):
    """
    This funciton compares the list of names from one dataframe to another and updates where the names do not match 
    so they will merge correctly.
    
    Parameters:
        df1: first dataframe to compare to; this should be the dataframe on the left that you will join with when 
        they merge.
        df2: second dataframe, should be the dataframe on the right for the later merge.
        
    Returns:
        Updated name list for df2 that should match df1.
    """
    
    df_players = list(df1['Player'])    
    for i in range(len(df2['Player'])):
        name = df2['Player'][i]
        split_name = name.split(' ')
        if (split_name[0] + ' ' + split_name[1]) in df_players:
            df2['Player'][i] = split_name[0] + ' ' + split_name[1]
        else:
            df2['Player'][i] = name

In [4]:
def name_update(dataframe, column):
    """
    This function updates a column in a dataframe by stripping any excess spaces surrounding the observation.
    
    Parameters:
        dataframe: the dataframe you want to update.
        column: the column you want to update.
    
    Returns:
        Updated dataframe column.    
    """
    dataframe[column] = dataframe[column].map(lambda x: x.strip())

In [5]:
def update_int_dtype(dataframe, column_list):
    """
    This function updates the data type for a list of columns to integer.
    
    Parameters:
        dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.
        
    Returns:
        Updated dataframe column data types to integer.   
    """
    for column in column_list:
        dataframe[column] = dataframe[column].map(lambda x: int(x))

In [6]:
def update_float_dtype(dataframe, column_list):
    """
    This function takes a dataframe object and converts it to a float.
    
    Parameters:
        dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.
    
    Returns:
        Updated dataframe column data types to a float.  
    """
    for column in column_list:
        for i in range(len(dataframe[column])):
            item = dataframe[column][i]
            if item == '':
                dataframe[column][i] = '0.0'
            else:
                dataframe[column][i] = item 
            updated_item = str(dataframe[column][i])
            ones, tenths = updated_item.split('.')
            ones = int(ones)
            tenths = int(tenths) * .1
            dataframe[column][i] = ones + tenths
        dataframe[column] = dataframe[column].map(lambda x: float(x))

In [7]:
def fill_blanks(dataframe, column_list):
    """
    This function fills blank values with 0. This should be used for a column that will be an integer.
    
    Parameters:
        dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.    
    
    Returns:
        Updated dataframe columns with filled values.      
    """
    for column in column_list:
        for i in range(len(dataframe[column])):
            item = dataframe[column][i]
            if item == '':
                dataframe[column][i] = 0
            else:
                dataframe[column][i] = item

In [8]:
def remove_comma(dataframe, column_list):
    """
    This function removes the comma from a value in a column.
    
    Parameters:
        dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.
        
    Returns:
        Updated dataframe columns less any commas that might appear.    
    """  
    for column in column_list:
        for i in range(len(dataframe[column])):
            item = dataframe[column][i]
            if len(item.split(',')) > 1:
                one, two = item.split(',')
                dataframe[column][i] = one + two

In [9]:
def adjust_float(dataframe, column_list):
    """
    This function adjusts values in a column to be workable with the 'update_float_dtype' function.
    
    Parameters:
        dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.
        
    Returns:
        Updated dataframe columns with objects with a decimal point value behind it.    
    """
    for column in column_list:
        for i in range(len(dataframe[column])):
            item = dataframe[column][i]
            if len(item) < 3:
                dataframe[column][i] = item + '.0'
            else:
                dataframe[column][i] = item

In [10]:
def update_percent(dataframe, column):
    """
    This function updates a column with a percent sign so it can be manipulated into a float with the 
    'update_float_dtype' function.
    
    Parameters:
       dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.
        
    Returns:
        Updated dataframe column with no percent sign and added decimal point where needed.     
    """
    dataframe[column] = dataframe[column].map(lambda x: x.strip('%'))
    for i in range(len(dataframe[column])):
        item = dataframe[column][i]
        if len(item) < 3:
            dataframe[column][i] = item + '.0'
        else:
            dataframe[column][i] = item

In [11]:
def replace_dash(dataframe, column_list, dash):
    """
    This function replaces a - in a dataframe so datatypes can then be updated.
    
    Parameters:
       dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.
        dash: set to a value in a specific column in the dataframe.
        
    Returns:
        Updated dataframe column with no percent sign and added decimal point where needed. 
    """
    for column in column_list:
        for i in range(len(dataframe[column])):
            item = dataframe[column][i]
            if item == dash:
                dataframe[column][i] = '0'
            else:
                dataframe[column][i] = item

## Read in Collected Data

In [44]:
df = pd.read_pickle('player_stats')
fantasy_pts = pd.read_pickle('fantasy_weeks')
defense = pd.read_pickle('defense_data')
kicking = pd.read_pickle('kicking_data')

## Merge Last Week Fantasy Pts with this Week

In [45]:
no_games = 11

In [46]:
lastwk = pd.read_pickle('fantweeks_1_' + str(no_games - 1))
lastwk.head()

,Player,Team,Position,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7,Week_8,Week_9,Week_10
0,Kyler Murray,ARI,QB,271.3,27.3,33.1,24.7,23.1,27.3,28.9,38.1,NaN,37.9,30.9
1,Josh Allen,BUF,QB,249.0,28.2,34.5,32.2,25.4,18.3,16.1,16.4,13.5,36.0,28.4
2,Russell Wilson,SEA,QB,248.0,31.8,34.4,36.8,21.9,25.5,NaN,32.9,28.7,24.1,11.9
3,Patrick Mahomes II,KC,QB,239.5,20.4,27.5,40.0,20.2,30.7,20.6,12.0,36.6,30.9,NaN
4,Aaron Rodgers,GB,QB,214.0,30.8,18.2,24.5,29.6,NaN,5.8,27.3,22.5,28.9,26.4


In [47]:
fantasy_pts = pd.merge(fantasy_pts, lastwk, 'left', on='Player')
fantasy_pts.head()

,Player,Team_x,Position_x,TTL_x,Week_11,Team_y,Position_y,TTL_y,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7,Week_8,Week_9,Week_10
0,Kyler Murray,ARI,QB,291.6,20.3,ARI,QB,271.3,27.3,33.1,24.7,23.1,27.3,28.9,38.1,NaN,37.9,30.9
1,Russell Wilson,SEA,QB,268.1,20.1,SEA,QB,248.0,31.8,34.4,36.8,21.9,25.5,NaN,32.9,28.7,24.1,11.9
2,Patrick Mahomes II,KC,QB,262.0,22.5,KC,QB,239.5,20.4,27.5,40.0,20.2,30.7,20.6,12.0,36.6,30.9,NaN
3,Josh Allen,BUF,QB,249.0,NaN,BUF,QB,249.0,28.2,34.5,32.2,25.4,18.3,16.1,16.4,13.5,36.0,28.4
4,Aaron Rodgers,GB,QB,236.7,22.7,GB,QB,214.0,30.8,18.2,24.5,29.6,NaN,5.8,27.3,22.5,28.9,26.4


In [48]:
fantasy_pts.drop(columns=['Team_y', 'Position_y', 'TTL_y'], inplace=True)
fantasy_pts.rename(columns = {'Team_x': 'Team', 'Position_x': 'Position', 'TTL_x': 'TTL'}, inplace = True)
fantasy_pts.columns

Index(['Player', 'Team', 'Position', 'TTL', 'Week_11', 'Week_1', 'Week_2',
       'Week_3', 'Week_4', 'Week_5', 'Week_6', 'Week_7', 'Week_8', 'Week_9',
       'Week_10'],
      dtype='object')

In [49]:
fantasy_pts = fantasy_pts[['Player', 'Team', 'Position', 'TTL', 'Week_1', 'Week_2', 'Week_3', 'Week_4', 'Week_5', 
                           'Week_6', 'Week_7', 'Week_8', 'Week_9', 'Week_10', 'Week_11']]
fantasy_pts.head()

,Player,Team,Position,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7,Week_8,Week_9,Week_10,Week_11
0,Kyler Murray,ARI,QB,291.6,27.3,33.1,24.7,23.1,27.3,28.9,38.1,NaN,37.9,30.9,20.3
1,Russell Wilson,SEA,QB,268.1,31.8,34.4,36.8,21.9,25.5,NaN,32.9,28.7,24.1,11.9,20.1
2,Patrick Mahomes II,KC,QB,262.0,20.4,27.5,40.0,20.2,30.7,20.6,12.0,36.6,30.9,NaN,22.5
3,Josh Allen,BUF,QB,249.0,28.2,34.5,32.2,25.4,18.3,16.1,16.4,13.5,36.0,28.4,NaN
4,Aaron Rodgers,GB,QB,236.7,30.8,18.2,24.5,29.6,NaN,5.8,27.3,22.5,28.9,26.4,22.7


In [50]:
print(fantasy_pts.shape)

(665, 15)


### Pickle Combined DataFrame for Next Week

In [51]:
fantasy_pts.to_pickle('fantweeks_1_' + str(no_games))

## Merge Offensive Players & Weekly Fantasy Pts

In [52]:
print(df.shape)
df.head()

(577, 28)


,Player,Team,Position,Age,Games,GamesStarted,CompletedPasses,PassesAttempted,PassingYds,PassingTDs,Interceptions,RushingAttempts,RushingYds,RushingYdspAtt,RushingTDs,Targeted,Receptions,ReceivingYds,YdspReception,ReceivingTDs,Fumbles,LostFumbles,TtlTDs,TwoPTConversions,TwoPTConversionPasses,FDFantasyPts,PositionRank,OverallRank
0,Dalvin Cook,MIN,RB,25,9,9,0,0,0,0,0,201,1069,5.32,13,30,25,234,9.36,1,3,2,14,3,,228.8,1,1
1,Alvin Kamara,NOR,RB,25,10,6,0,0,0,0,0,117,531,4.54,8,81,67,648,9.67,4,1,0,12,,,223.4,2,2
2,Kyler Murray,ARI,QB,23,10,10,241,353,2644,19,8,92,619,6.73,10,0,0,0,,0,4,2,10,,,291.7,1,3
3,Derrick Henry,TEN,RB,26,10,10,0,0,0,0,0,229,1079,4.71,9,22,12,86,7.17,0,0,0,9,,,176.5,3,4
4,Patrick Mahomes,KAN,QB,25,10,10,254,374,3035,27,2,39,187,4.79,2,0,0,0,,0,1,0,2,,2,262.1,2,5


In [53]:
print(fantasy_pts.shape)
fantasy_pts.head()

(665, 15)


,Player,Team,Position,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7,Week_8,Week_9,Week_10,Week_11
0,Kyler Murray,ARI,QB,291.6,27.3,33.1,24.7,23.1,27.3,28.9,38.1,NaN,37.9,30.9,20.3
1,Russell Wilson,SEA,QB,268.1,31.8,34.4,36.8,21.9,25.5,NaN,32.9,28.7,24.1,11.9,20.1
2,Patrick Mahomes II,KC,QB,262.0,20.4,27.5,40.0,20.2,30.7,20.6,12.0,36.6,30.9,NaN,22.5
3,Josh Allen,BUF,QB,249.0,28.2,34.5,32.2,25.4,18.3,16.1,16.4,13.5,36.0,28.4,NaN
4,Aaron Rodgers,GB,QB,236.7,30.8,18.2,24.5,29.6,NaN,5.8,27.3,22.5,28.9,26.4,22.7


In [54]:
check_names(df, fantasy_pts)

In [55]:
name_update(df, 'Player')
name_update(fantasy_pts, 'Player')

In [56]:
offense = pd.merge(df, fantasy_pts, 'left', on='Player')

In [57]:
offense.drop(columns = ['Team_y', 'Position_y'], inplace = True)
offense.rename(columns = {'Team_x': 'Team', 'Position_x': 'Position'}, inplace = True)

In [58]:
print(offense.shape)
offense.head(10)

(577, 40)


,Player,Team,Position,Age,Games,GamesStarted,CompletedPasses,PassesAttempted,PassingYds,PassingTDs,Interceptions,RushingAttempts,RushingYds,RushingYdspAtt,RushingTDs,Targeted,Receptions,ReceivingYds,YdspReception,ReceivingTDs,Fumbles,LostFumbles,TtlTDs,TwoPTConversions,TwoPTConversionPasses,FDFantasyPts,PositionRank,OverallRank,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7,Week_8,Week_9,Week_10,Week_11
0,Dalvin Cook,MIN,RB,25,9,9,0,0,0,0,0,201,1069,5.32,13,30,25,234,9.36,1,3,2,14,3,,228.8,1,1,216.3,20.8,15.1,23.9,26.6,14.9,NaN,NaN,46.6,37.2,11.2,20.0
1,Alvin Kamara,NOR,RB,25,10,6,0,0,0,0,0,117,531,4.54,8,81,67,648,9.67,4,1,0,12,,,223.4,2,2,189.9,18.7,29.4,31.7,17.9,11.9,NaN,14.8,16.3,10.9,27.8,10.5
2,Kyler Murray,ARI,QB,23,10,10,241,353,2644,19,8,92,619,6.73,10,0,0,0,,0,4,2,10,,,291.7,1,3,291.6,27.3,33.1,24.7,23.1,27.3,28.9,38.1,NaN,37.9,30.9,20.3
3,Derrick Henry,TEN,RB,26,10,10,0,0,0,0,0,229,1079,4.71,9,22,12,86,7.17,0,0,0,9,,,176.5,3,4,170.5,13.1,8.4,25.0,NaN,18.3,38.4,13.2,17.2,6.8,10.9,19.2
4,Patrick Mahomes,KAN,QB,25,10,10,254,374,3035,27,2,39,187,4.79,2,0,0,0,,0,1,0,2,,2,262.1,2,5,262.0,20.4,27.5,40.0,20.2,30.7,20.6,12.0,36.6,30.9,NaN,22.5
5,Russell Wilson,SEA,QB,32,10,10,256,362,2986,30,10,55,367,6.67,1,0,0,0,,0,6,3,1,,1,268.1,3,6,268.1,31.8,34.4,36.8,21.9,25.5,NaN,32.9,28.7,24.1,11.9,20.1
6,Travis Kelce,KAN,TE,31,10,10,1,1,4,0,0,0,0,,0,90,66,896,13.58,7,1,1,7,,,162.8,1,7,129.8,11.0,15.0,8.7,7.0,16.8,16.5,3.1,16.9,15.9,NaN,18.9
7,Tyreek Hill,KAN,WR,26,10,10,0,0,0,0,0,10,60,6.00,1,86,55,752,13.67,10,1,0,11,,,174.7,1,8,147.2,10.6,16.8,16.2,12.4,15.3,2.5,11.5,21.8,24.1,NaN,16.0
8,Josh Allen,BUF,QB,24,10,10,249,364,2871,21,7,72,279,3.88,5,1,1,12,12.00,1,4,4,6,,,249.4,4,9,249.0,28.2,34.5,32.2,25.4,18.3,16.1,16.4,13.5,36.0,28.4,NaN
9,Davante Adams,GNB,WR,28,8,8,0,0,0,0,0,0,0,,0,89,68,847,12.46,10,1,1,10,,,176.7,2,10,142.7,27.6,3.6,NaN,NaN,NaN,6.1,31.6,23.3,23.3,10.6,16.6


In [59]:
offense.isnull().sum()

Player                     0
Team                       0
Position                   0
Age                        0
Games                      0
GamesStarted               0
CompletedPasses            0
PassesAttempted            0
PassingYds                 0
PassingTDs                 0
Interceptions              0
RushingAttempts            0
RushingYds                 0
RushingYdspAtt             0
RushingTDs                 0
Targeted                   0
Receptions                 0
ReceivingYds               0
YdspReception              0
ReceivingTDs               0
Fumbles                    0
LostFumbles                0
TtlTDs                     0
TwoPTConversions           0
TwoPTConversionPasses      0
FDFantasyPts               0
PositionRank               0
OverallRank                0
TTL                       22
Week_1                   168
Week_2                   170
Week_3                   172
Week_4                   198
Week_5                   228
Week_6        

In [60]:
offense.fillna('0.0', inplace = True)

In [61]:
off_integers = ['Age', 'Games', 'GamesStarted', 'CompletedPasses', 'PassesAttempted', 'PassingYds', 'PassingTDs', 
            'Interceptions', 'RushingAttempts', 'RushingYds', 'RushingTDs', 'Targeted', 'Receptions', 
            'ReceivingYds', 'ReceivingTDs', 'Fumbles', 'LostFumbles', 'TtlTDs']

off_floats = ['RushingYdspAtt', 'YdspReception', 'FDFantasyPts', 'TTL','Week_1', 'Week_2', 'Week_3', 'Week_4', 
              'Week_5', 'Week_6', 'Week_7', 'Week_8', 'Week_9', 'Week_10', 'Week_11']

In [62]:
update_int_dtype(offense, off_integers)

In [63]:
two_pts = ['TwoPTConversions', 'TwoPTConversionPasses']
fill_blanks(offense, two_pts)
update_int_dtype(offense, two_pts)

In [64]:
update_float_dtype(offense, off_floats)

In [65]:
offense.head()

,Player,Team,Position,Age,Games,GamesStarted,CompletedPasses,PassesAttempted,PassingYds,PassingTDs,Interceptions,RushingAttempts,RushingYds,RushingYdspAtt,RushingTDs,Targeted,Receptions,ReceivingYds,YdspReception,ReceivingTDs,Fumbles,LostFumbles,TtlTDs,TwoPTConversions,TwoPTConversionPasses,FDFantasyPts,PositionRank,OverallRank,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7,Week_8,Week_9,Week_10,Week_11
0,Dalvin Cook,MIN,RB,25,9,9,0,0,0,0,0,201,1069,8.2,13,30,25,234,12.6,1,3,2,14,3,0,228.8,1,1,216.3,20.8,15.1,23.9,26.6,14.9,0.0,0.0,46.6,37.2,11.2,20.0
1,Alvin Kamara,NOR,RB,25,10,6,0,0,0,0,0,117,531,9.4,8,81,67,648,15.7,4,1,0,12,0,0,223.4,2,2,189.9,18.7,29.4,31.7,17.9,11.9,0.0,14.8,16.3,10.9,27.8,10.5
2,Kyler Murray,ARI,QB,23,10,10,241,353,2644,19,8,92,619,13.3,10,0,0,0,0.0,0,4,2,10,0,0,291.7,1,3,291.6,27.3,33.1,24.7,23.1,27.3,28.9,38.1,0.0,37.9,30.9,20.3
3,Derrick Henry,TEN,RB,26,10,10,0,0,0,0,0,229,1079,11.1,9,22,12,86,8.7,0,0,0,9,0,0,176.5,3,4,170.5,13.1,8.4,25.0,0.0,18.3,38.4,13.2,17.2,6.8,10.9,19.2
4,Patrick Mahomes,KAN,QB,25,10,10,254,374,3035,27,2,39,187,11.9,2,0,0,0,0.0,0,1,0,2,0,2,262.1,2,5,262.0,20.4,27.5,40.0,20.2,30.7,20.6,12.0,36.6,30.9,0.0,22.5


In [66]:
team_dict = {'LAR': 'Los Angeles Rams', 
             'SEA': 'Seattle Seahawks', 
             'BAL': 'Baltimore Ravens', 
             'KAN': 'Kansas City Chiefs', 
             'JAX': 'Jacksonville Jaguars', 
             'DET': 'Detroit Lions', 
             'PIT': 'Pittsburgh Steelers', 
             'MIN': 'Minnesota Vikings', 
             'CHI': 'Chicago Bears', 
             'ATL': 'Atlanta Falcons', 
             'BUF': 'Buffalo Bills', 
             'GNB': 'Green Bay Packers', 
             'MIA': 'Miami Dolphins', 
             'CLE': 'Cleveland Browns', 
             'CAR': 'Carolina Panthers', 
             'DAL': 'Dallas Cowboys', 
             'ARI': 'Arizona Cardinals', 
             'HOU': 'Houston Texans', 
             'NYG': 'New York Giants', 
             'WAS': 'Washington Football Team', 
             'IND': 'Indianapolis Colts', 
             'LVR': 'Las Vegas Raiders', 
             'DEN': 'Denver Broncos', 
             'TEN': 'Tennessee Titans', 
             'NWE': 'New England Patriots', 
             'CIN': 'Cincinnati Bengals', 
             'NYJ': 'New York Jets', 
             'LAC': 'Los Angeles Chargers', 
             'PHI': 'Philadelphia Eagles', 
             'TAM': 'Tampa Bay Buccaneers', 
             'NOR': 'New Orleans Saints', 
             'SFO': 'San Francisco 49ers'}

team_names = pd.DataFrame()
team_names['Team'] = list(team_dict.keys())
team_names['Long_Name'] = list(team_dict.values())
team_names.head()

,Team,Long_Name
0,LAR,Los Angeles Rams
1,SEA,Seattle Seahawks
2,BAL,Baltimore Ravens
3,KAN,Kansas City Chiefs
4,JAX,Jacksonville Jaguars


In [67]:
offense.dtypes

Player                    object
Team                      object
Position                  object
Age                        int64
Games                      int64
GamesStarted               int64
CompletedPasses            int64
PassesAttempted            int64
PassingYds                 int64
PassingTDs                 int64
Interceptions              int64
RushingAttempts            int64
RushingYds                 int64
RushingYdspAtt           float64
RushingTDs                 int64
Targeted                   int64
Receptions                 int64
ReceivingYds               int64
YdspReception            float64
ReceivingTDs               int64
Fumbles                    int64
LostFumbles                int64
TtlTDs                     int64
TwoPTConversions           int64
TwoPTConversionPasses      int64
FDFantasyPts             float64
PositionRank              object
OverallRank               object
TTL                      float64
Week_1                   float64
Week_2    

## Defense Stats

In [68]:
defense.head()

,Team,GP,Ttl_Pts_Allowed,Ttl_Offense_Plays_Allowed,Yds_p_Play,Ttl_Yds,Rushing_Att,Rushing_Yds,Rushing_Yds_p_Att,Rushing_TDs,Passing_Att,Passing_Yds_p_Att,Completions,Yds_p_Completion,Passing_Yds,Passing_TDs,RZ_Att,RZ_TD,RZ_Percent,Ttl_Turnovers,Interceptions,Fumbles,Sacks
0,Pittsburgh Steelers,10,174,621,4.9,"3,069",238,"1,034",4.3,5,345,5.3,189,12.1,"2,035",15,25,14,56%,21,15,6,38
1,Los Angeles Rams,10,192,625,4.7,"2,919",232,913,3.9,8,361,5.1,227,9.7,"2,006",11,29,18,62.1%,15,10,5,32
2,Baltimore Ravens,10,195,638,5.2,"3,330",253,"1,160",4.6,8,358,5.6,232,10.2,"2,170",14,26,18,69.2%,15,5,10,27
3,Miami Dolphins,10,202,657,5.8,"3,806",277,"1,342",4.8,13,358,6.5,224,11.9,"2,464",13,32,21,65.6%,17,9,8,22
4,Indianapolis Colts,10,208,598,5,"2,981",252,892,3.5,8,324,6,205,11,"2,089",14,29,19,65.5%,17,12,5,22


In [69]:
def_fantasy = fantasy_pts[fantasy_pts['Position'] == 'DST']

In [83]:
defense_df = pd.merge(defense, def_fantasy, 'left', left_on = 'Team', right_on = 'Player')

In [84]:
defense_df.head()

,Team_x,GP,Ttl_Pts_Allowed,Ttl_Offense_Plays_Allowed,Yds_p_Play,Ttl_Yds,Rushing_Att,Rushing_Yds,Rushing_Yds_p_Att,Rushing_TDs,Passing_Att,Passing_Yds_p_Att,Completions,Yds_p_Completion,Passing_Yds,Passing_TDs,RZ_Att,RZ_TD,RZ_Percent,Ttl_Turnovers,Interceptions,Fumbles,Sacks,Player,Team_y,Position,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7,Week_8,Week_9,Week_10,Week_11
0,Pittsburgh Steelers,10,174,621,4.9,"3,069",238,"1,034",4.3,5,345,5.3,189,12.1,"2,035",15,25,14,56%,21,15,6,38,Pittsburgh Steelers,PIT,DST,110.0,8.0,13.0,7.0,10.0,8.0,18.0,2.0,18.0,7.0,12.0,17.0
1,Los Angeles Rams,10,192,625,4.7,"2,919",232,913,3.9,8,361,5.1,227,9.7,"2,006",11,29,18,62.1%,15,10,5,32,Los Angeles Rams,LAR,DST,76.0,4.0,7.0,4.0,11.0,12.0,0.0,15.0,5.0,NaN,13.0,5.0
2,Baltimore Ravens,10,195,638,5.2,"3,330",253,"1,160",4.6,8,358,5.6,232,10.2,"2,170",14,26,18,69.2%,15,5,10,27,Baltimore Ravens,BAL,DST,98.0,15.0,15.0,1.0,6.0,26.0,7.0,NaN,4.0,14.0,1.0,3.0
3,Miami Dolphins,10,202,657,5.8,"3,806",277,"1,342",4.8,13,358,6.5,224,11.9,"2,464",13,32,21,65.6%,17,9,8,22,Miami Dolphins,MIA,DST,88.0,2.0,2.0,12.0,3.0,12.0,15.0,NaN,17.0,8.0,4.0,5.0
4,Indianapolis Colts,10,208,598,5,"2,981",252,892,3.5,8,324,6,205,11,"2,089",14,29,19,65.5%,17,12,5,22,Indianapolis Colts,IND,DST,106.0,4.0,15.0,26.0,7.0,5.0,4.0,NaN,15.0,5.0,10.0,8.0


In [85]:
defense_df.drop(columns=['Player', 'Team_y', 'Position'], inplace=True)
defense_df.rename(columns={'Team_x': 'Team'}, inplace=True)

In [86]:
defense_df.shape

(32, 35)

In [87]:
defense_df.isnull().sum()

Team                         0
GP                           0
Ttl_Pts_Allowed              0
Ttl_Offense_Plays_Allowed    0
Yds_p_Play                   0
Ttl_Yds                      0
Rushing_Att                  0
Rushing_Yds                  0
Rushing_Yds_p_Att            0
Rushing_TDs                  0
Passing_Att                  0
Passing_Yds_p_Att            0
Completions                  0
Yds_p_Completion             0
Passing_Yds                  0
Passing_TDs                  0
RZ_Att                       0
RZ_TD                        0
RZ_Percent                   0
Ttl_Turnovers                0
Interceptions                0
Fumbles                      0
Sacks                        0
TTL                          0
Week_1                       0
Week_2                       0
Week_3                       0
Week_4                       0
Week_5                       4
Week_6                       4
Week_7                       4
Week_8                       4
Week_9  

In [88]:
defense_df

,Team,GP,Ttl_Pts_Allowed,Ttl_Offense_Plays_Allowed,Yds_p_Play,Ttl_Yds,Rushing_Att,Rushing_Yds,Rushing_Yds_p_Att,Rushing_TDs,Passing_Att,Passing_Yds_p_Att,Completions,Yds_p_Completion,Passing_Yds,Passing_TDs,RZ_Att,RZ_TD,RZ_Percent,Ttl_Turnovers,Interceptions,Fumbles,Sacks,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7,Week_8,Week_9,Week_10,Week_11
0,Pittsburgh Steelers,10,174,621,4.9,"3,069",238,"1,034",4.3,5,345,5.3,189,12.1,"2,035",15,25,14,56%,21,15,6,38,110.0,8.0,13.0,7.0,10.0,8.0,18.0,2.0,18.0,7.0,12.0,17.0
1,Los Angeles Rams,10,192,625,4.7,"2,919",232,913,3.9,8,361,5.1,227,9.7,"2,006",11,29,18,62.1%,15,10,5,32,76.0,4.0,7.0,4.0,11.0,12.0,0.0,15.0,5.0,NaN,13.0,5.0
2,Baltimore Ravens,10,195,638,5.2,"3,330",253,"1,160",4.6,8,358,5.6,232,10.2,"2,170",14,26,18,69.2%,15,5,10,27,98.0,15.0,15.0,1.0,6.0,26.0,7.0,NaN,4.0,14.0,1.0,3.0
3,Miami Dolphins,10,202,657,5.8,"3,806",277,"1,342",4.8,13,358,6.5,224,11.9,"2,464",13,32,21,65.6%,17,9,8,22,88.0,2.0,2.0,12.0,3.0,12.0,15.0,NaN,17.0,8.0,4.0,5.0
4,Indianapolis Colts,10,208,598,5,"2,981",252,892,3.5,8,324,6,205,11,"2,089",14,29,19,65.5%,17,12,5,22,106.0,4.0,15.0,26.0,7.0,5.0,4.0,NaN,15.0,5.0,10.0,8.0
5,Chicago Bears,10,209,658,5.2,"3,401",287,"1,151",4,6,350,6.1,214,11.1,"2,250",12,34,15,44.1%,11,6,5,21,64.0,3.0,12.0,4.0,2.0,6.0,11.0,9.0,1.0,4.0,12.0,NaN
6,Kansas City Chiefs,10,214,646,5.5,"3,551",288,"1,335",4.6,10,339,6.2,215,10.8,"2,216",14,29,22,75.9%,15,10,5,19,74.0,7.0,5.0,7.0,20.0,-1.0,3.0,18.0,7.0,1.0,NaN,1.0
7,New Orleans Saints,10,222,608,5,"3,022",222,743,3.3,5,354,5.9,225,11.1,"2,279",20,29,21,72.4%,15,11,4,32,77.0,15.0,4.0,-3.0,4.0,3.0,NaN,1.0,7.0,16.0,14.0,16.0
8,Washington Football Team,10,227,619,5.1,"3,158",279,"1,204",4.3,9,308,5.7,197,11,"1,954",15,28,15,53.6%,11,9,2,32,64.0,15.0,4.0,1.0,4.0,2.0,4.0,17.0,NaN,5.0,0.0,12.0
9,San Francisco 49ers,10,234,603,5.2,"3,159",271,"1,070",3.9,7,314,6.3,204,10.8,"2,089",17,26,16,61.5%,12,8,4,18,50.0,4.0,5.0,12.0,6.0,-2.0,3.0,17.0,-2.0,0.0,7.0,NaN


In [89]:
defense_df.fillna('0.0', inplace=True)

In [90]:
def_int = ['GP', 'Ttl_Pts_Allowed', 'Ttl_Offense_Plays_Allowed', 'Ttl_Yds', 'Rushing_Att', 'Rushing_Yds', 
           'Rushing_TDs', 'Passing_Att', 'Completions', 'Passing_Yds', 'Passing_TDs', 'RZ_Att', 'RZ_TD', 
           'Ttl_Turnovers', 'Interceptions', 'Fumbles', 'Sacks']

def_floats = ['Yds_p_Play', 'TTL', 'Week_1', 'Rushing_Yds_p_Att', 'Passing_Yds_p_Att', 'Yds_p_Completion',
              'Week_2', 'Week_3', 'Week_4', 'Week_5', 'Week_6', 'Week_7', 'Week_8', 'Week_9', 'Week_10', 'Week_11']



In [91]:
remove_comma(defense_df, def_int)

In [92]:
update_int_dtype(defense_df, def_int)

In [93]:
adjust_float(defense_df, def_floats)

In [94]:
update_float_dtype(defense_df, def_floats)

In [95]:
update_percent(defense_df, 'RZ_Percent')

In [96]:
rz = ['RZ_Percent']
update_float_dtype(defense_df, rz)

In [97]:
defense_df.head()

,Team,GP,Ttl_Pts_Allowed,Ttl_Offense_Plays_Allowed,Yds_p_Play,Ttl_Yds,Rushing_Att,Rushing_Yds,Rushing_Yds_p_Att,Rushing_TDs,Passing_Att,Passing_Yds_p_Att,Completions,Yds_p_Completion,Passing_Yds,Passing_TDs,RZ_Att,RZ_TD,RZ_Percent,Ttl_Turnovers,Interceptions,Fumbles,Sacks,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7,Week_8,Week_9,Week_10,Week_11
0,Pittsburgh Steelers,10,174,621,4.9,3069,238,1034,4.3,5,345,5.3,189,12.1,2035,15,25,14,56.0,21,15,6,38,110.0,8.0,13.0,7.0,10.0,8.0,18.0,2.0,18.0,7.0,12.0,17.0
1,Los Angeles Rams,10,192,625,4.7,2919,232,913,3.9,8,361,5.1,227,9.7,2006,11,29,18,62.1,15,10,5,32,76.0,4.0,7.0,4.0,11.0,12.0,0.0,15.0,5.0,0.0,13.0,5.0
2,Baltimore Ravens,10,195,638,5.2,3330,253,1160,4.6,8,358,5.6,232,10.2,2170,14,26,18,69.2,15,5,10,27,98.0,15.0,15.0,1.0,6.0,26.0,7.0,0.0,4.0,14.0,1.0,3.0
3,Miami Dolphins,10,202,657,5.8,3806,277,1342,4.8,13,358,6.5,224,11.9,2464,13,32,21,65.6,17,9,8,22,88.0,2.0,2.0,12.0,3.0,12.0,15.0,0.0,17.0,8.0,4.0,5.0
4,Indianapolis Colts,10,208,598,5.0,2981,252,892,3.5,8,324,6.0,205,11.0,2089,14,29,19,65.5,17,12,5,22,106.0,4.0,15.0,26.0,7.0,5.0,4.0,0.0,15.0,5.0,10.0,8.0


In [98]:
defense_df.dtypes

Team                          object
GP                             int64
Ttl_Pts_Allowed                int64
Ttl_Offense_Plays_Allowed      int64
Yds_p_Play                   float64
Ttl_Yds                        int64
Rushing_Att                    int64
Rushing_Yds                    int64
Rushing_Yds_p_Att            float64
Rushing_TDs                    int64
Passing_Att                    int64
Passing_Yds_p_Att            float64
Completions                    int64
Yds_p_Completion             float64
Passing_Yds                    int64
Passing_TDs                    int64
RZ_Att                         int64
RZ_TD                          int64
RZ_Percent                   float64
Ttl_Turnovers                  int64
Interceptions                  int64
Fumbles                        int64
Sacks                          int64
TTL                          float64
Week_1                       float64
Week_2                       float64
Week_3                       float64
W

## Kicking Stats

In [99]:
kicking.head()

,PLAYER,POSITION,TEAM,GP,FGM_A,FG%,LNG,_1_19,_20_29,_30_39,_40_49,OVER50,XPM_A,XP%,PTS
0,Ryan Succop,K,TB,11,23,91.3,50,0-0,6-6,8-8,6-7,1-2,35,94.3,96
1,Rodrigo Blankenship,K,IND,10,26,88.5,44,0-0,5-5,10-11,8-9,0-1,27,92.6,94
2,Daniel Carlson,K,LV,10,23,91.3,54,0-0,9-9,6-6,2-4,4-4,32,96.9,94
3,Wil Lutz,K,NO,10,21,90.5,53,0-0,5-6,8-8,5-5,1-2,34,100.0,91
4,Younghoe Koo,K,ATL,9,25,96.0,54,0-0,6-6,6-6,7-8,5-5,21,85.7,90


In [100]:
kicking_df = pd.merge(kicking, fantasy_pts, left_on = 'PLAYER', right_on = 'Player')
kicking_df.drop(columns = ['Player', 'Team', 'Position'], inplace = True)
kicking_df

,PLAYER,POSITION,TEAM,GP,FGM_A,FG%,LNG,_1_19,_20_29,_30_39,_40_49,OVER50,XPM_A,XP%,PTS,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7,Week_8,Week_9,Week_10,Week_11
0,Ryan Succop,K,TB,11,23,91.3,50,0-0,6-6,8-8,6-7,1-2,35,94.3,96,104.0,5.0,7.0,9.0,8.0,14.0,10.0,9.0,15.0,4.0,17.0,6.0
1,Rodrigo Blankenship,K,IND,10,26,88.5,44,0-0,5-5,10-11,8-9,0-1,27,92.6,94,102.0,8.0,15.0,12.0,14.0,11.0,8.0,NaN,3.0,5.0,11.0,15.0
2,Daniel Carlson,K,LV,10,23,91.3,54,0-0,9-9,6-6,2-4,4-4,32,96.9,94,104.0,12.0,12.0,8.0,13.0,11.0,NaN,9.0,10.0,7.0,15.0,7.0
3,Wil Lutz,K,NO,10,21,90.5,53,0-0,5-6,8-8,5-5,1-2,34,100.0,91,98.0,10.0,6.0,13.0,5.0,15.0,NaN,11.0,14.0,8.0,10.0,6.0
4,Younghoe Koo,K,ATL,9,25,96.0,54,0-0,6-6,6-6,7-8,5-5,21,85.7,90,107.0,9.0,17.0,8.0,NaN,13.0,20.0,2.0,13.0,12.0,NaN,13.0
5,Jason Sanders,K,MIA,10,23,95.7,56,0-0,5-5,3-3,8-9,6-6,24,100.0,90,110.0,4.0,11.0,7.0,18.0,22.0,6.0,NaN,4.0,14.0,14.0,10.0
6,Joey Slye,K,CAR,11,27,81.5,56,0-0,8-8,7-7,6-7,1-5,25,92.0,89,97.0,12.0,5.0,15.0,7.0,11.0,11.0,7.0,5.0,8.0,6.0,10.0
7,Tyler Bass,K,BUF,10,25,76.0,58,1-1,4-4,4-7,6-7,4-6,30,96.7,86,100.0,9.0,7.0,5.0,6.0,5.0,6.0,23.0,6.0,15.0,18.0,NaN
8,Justin Tucker,K,BAL,10,20,95.0,55,0-0,5-5,4-4,8-8,2-3,29,100.0,86,98.0,9.0,16.0,9.0,8.0,10.0,16.0,NaN,8.0,7.0,5.0,10.0
9,Randy Bullock,K,CIN,10,24,83.3,55,0-0,3-3,8-10,6-7,3-4,22,95.5,81,93.0,10.0,13.0,12.0,17.0,3.0,12.0,10.0,7.0,NaN,4.0,5.0


In [101]:
dash = kicking_df['LNG'][40]
dash

'—'

In [102]:
kicking_df.fillna('0.0', inplace = True)

In [103]:
kick_int = ['GP', 'FGM_A', 'LNG', 'XPM_A', 'PTS']

kick_float = ['FG%', 'XP%', 'TTL', 'Week_1', 'Week_2', 'Week_3', 'Week_4', 'Week_5', 'Week_6', 'Week_7', 'Week_8', 
              'Week_9', 'Week_10', 'Week_11']


In [104]:
replace_dash(kicking_df, kick_int, dash)

In [105]:
replace_dash(kicking_df, kick_float, dash)

In [106]:
update_int_dtype(kicking_df, kick_int)

In [107]:
adjust_float(kicking_df, kick_float)

In [108]:
update_float_dtype(kicking_df, kick_float)

In [109]:
kicking_df.head()

,PLAYER,POSITION,TEAM,GP,FGM_A,FG%,LNG,_1_19,_20_29,_30_39,_40_49,OVER50,XPM_A,XP%,PTS,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7,Week_8,Week_9,Week_10,Week_11
0,Ryan Succop,K,TB,11,23,91.3,50,0-0,6-6,8-8,6-7,1-2,35,94.3,96,104.0,5.0,7.0,9.0,8.0,14.0,10.0,9.0,15.0,4.0,17.0,6.0
1,Rodrigo Blankenship,K,IND,10,26,88.5,44,0-0,5-5,10-11,8-9,0-1,27,92.6,94,102.0,8.0,15.0,12.0,14.0,11.0,8.0,0.0,3.0,5.0,11.0,15.0
2,Daniel Carlson,K,LV,10,23,91.3,54,0-0,9-9,6-6,2-4,4-4,32,96.9,94,104.0,12.0,12.0,8.0,13.0,11.0,0.0,9.0,10.0,7.0,15.0,7.0
3,Wil Lutz,K,NO,10,21,90.5,53,0-0,5-6,8-8,5-5,1-2,34,100.0,91,98.0,10.0,6.0,13.0,5.0,15.0,0.0,11.0,14.0,8.0,10.0,6.0
4,Younghoe Koo,K,ATL,9,25,96.0,54,0-0,6-6,6-6,7-8,5-5,21,85.7,90,107.0,9.0,17.0,8.0,0.0,13.0,20.0,2.0,13.0,12.0,0.0,13.0


In [110]:
kicking_df.dtypes

PLAYER       object
POSITION     object
TEAM         object
GP            int64
FGM_A         int64
FG%         float64
LNG           int64
_1_19        object
_20_29       object
_30_39       object
_40_49       object
OVER50       object
XPM_A         int64
XP%         float64
PTS           int64
TTL         float64
Week_1      float64
Week_2      float64
Week_3      float64
Week_4      float64
Week_5      float64
Week_6      float64
Week_7      float64
Week_8      float64
Week_9      float64
Week_10     float64
Week_11     float64
dtype: object

## Pickle Cleaned DataFrames

In [111]:
offense.to_pickle('players')
team_names.to_pickle('long_names')
defense_df.to_pickle('defense')
kicking_df.to_pickle('kicking')